In [1]:
library(tidyverse)
library(ggpubr)
library(GGally)
#library(Seurat)
library(ggvenn)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Loading required package: grid



In [2]:
DNA <- read_table2("../stat/dna.fq.stat",col_names=F) %>% arrange(X1) %>%rowwise()%>% mutate(X2 = X2/4,X1 = strsplit(X1,split = "/")[[1]][3]) 
Raw <- read_table2("../stat/raw.fq.stat",col_names=F) %>% arrange(X1) %>% mutate(X2 = X2/4) %>% select(X2)
RNA <- read_table2("../stat/rna.fq.stat",col_names=F) %>% arrange(X1) %>% mutate(X2 = X2/4) %>% select(X2)
raw_pairs <- read_table2("../stat/raw.pairs.stat",col_names=F) %>% arrange(X1) %>% select(X2)
pairs_dedup <- read_table2("../stat/pairs.dedup.stat",col_names=F) %>% arrange(X1) %>% select(X2)
pairs_c1 <- read_table2("../stat/pairs.c1.stat",col_names=F)%>% arrange(X1) %>% select(X2)
pairs_c12 <- read_table2("../stat/pairs.c12.stat",col_names=F)%>% arrange(X1) %>% select(X2)
inter_pairs_c12 <- read_table2(file = "../stat/inter.pairs.c12.stat",col_names = F) %>% arrange(X1) %>% select(X2)
RNAres <- read_table2("../result/RNA_Res/counts.gene.total.tsv")

featureStat <- cbind(as.data.frame(colSums(RNAres %>% select(-gene))),colSums(RNAres %>% select(-gene) != 0))
names(featureStat) <- c("UMIs","genes")
RNAresExon <- read_table2("../result/RNA_Res/counts.exon.total.tsv")

featureStatExon <- cbind(as.data.frame(colSums(RNAresExon %>% select(-gene))),colSums(RNAresExon %>% select(-gene) != 0))
names(featureStatExon) <- c("UMIs","genes")

#stat <- cbind(DNA,Raw,RNA,raw_pairs,pairs_dedup,pairs_c1,pairs_c12,inter_pairs_c12,featureStat)
#names(stat) <- c("cell_name","DNAreads","Rawreads","RNAreads","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","inter_pairs_clean2","UMIs_gene","genes_gene")

#rmsd <- read_table2("stat/rmsd.info",col_names = F) %>% arrange(X1) %>% select(X2)

#stat <- cbind(DNA,Raw,RNA,raw_pairs,pairs_dedup,pairs_c1,pairs_c12,inter_pairs_c12,featureStat,rmsd)
#names(stat) <- c("cell_name","DNAreads","Rawreads","RNAreads","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","inter_pairs_clean2","UMIs_gene","genes_gene","RMS_RMSD")


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [4]:
CT <- read_table2("../stat/ct.stat",col_names=F) %>% arrange(X1)  %>% mutate(X2 = X2/4) %>% select(X2)
CTfrag <- read_table2("../stat/ct.frag.stat",col_names=F) %>% arrange(X1) %>% select(X2)
temp <- cbind(DNA,Raw,RNA,CT,CTfrag,raw_pairs,pairs_dedup,pairs_c1,pairs_c12,inter_pairs_c12)
names(temp) <- c("cell_name","DNAreads","Rawreads","RNAreads","CTreads","CTfrag","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","inter_pairs_clean2")
stat <- featureStat %>% rownames_to_column("cell_name") %>% full_join(temp)
stat[is.na(stat)] <- 0


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


Joining, by = "cell_name"



In [6]:
stat  %>% mutate(RNAreadsRatio = RNAreads/Rawreads,
                 Rawreads = Rawreads * 300 /1000000000,
                 CTreadsRatio = CTreads / Rawreads,
                        pairsPerRead = raw_pairs/DNAreads,
                        pairsValidRatio = pairs_clean2/raw_pairs,
                        interPairsRatio = inter_pairs_clean2/pairs_clean2) 

cell_name,UMIs,genes,DNAreads,Rawreads,RNAreads,CTreads,CTfrag,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,inter_pairs_clean2,RNAreadsRatio,CTreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CSH003,4,4,33619998,10.086071,10,10631128,171596,6544821,827780,827780,751542,173376,2.974399e-07,1054040.56,0.1946705,0.11483003,0.2306937
CSH004,1,1,12734290,3.820340,12,364778,11592,2151197,593667,593667,544157,78059,9.423244e-07,95483.11,0.1689295,0.25295545,0.1434494
CSRH001,23009,5764,19702225,6.300845,1123984,658675,11058,3585754,858452,858452,774768,150783,5.351587e-02,104537.56,0.1819974,0.21606836,0.1946170
CSRH002,16769,5104,20736629,6.563592,988433,2587163,41769,3337646,846131,846131,760239,167603,4.517799e-02,394168.77,0.1609541,0.22777700,0.2204609
CSRH003,27723,6924,14725969,4.769725,1011589,1605349,56838,2593898,890894,890894,785218,135218,6.362562e-02,336570.56,0.1761445,0.30271738,0.1722044
CSRH004,29337,6293,27092911,8.732705,1759517,5832613,92143,4488776,950124,950124,860578,231213,6.044577e-02,667904.52,0.1656808,0.19171774,0.2686718
SCH001,7,7,48303107,14.491039,22,20208122,584617,7370382,952920,952920,859963,207471,4.554539e-07,1394525.44,0.1525861,0.11667821,0.2412557
SCH002,6,6,40054960,12.016605,28,12213488,187877,5919252,705548,705548,629246,171738,6.990327e-07,1016384.24,0.1477783,0.10630499,0.2729266
SCH003,7,7,51577238,15.473278,20,24107570,524712,8092219,835901,835901,740310,168690,3.877653e-07,1558013.12,0.1568952,0.09148418,0.2278640


In [8]:
RNAres %>% arrange(desc(CSRH001))

gene,CSH003,CSH004,CSRH001,CSRH002,CSRH003,CSRH004,SCH001,SCH002,SCH003,SCH004,SCRH001,SCRH002,SCRH003,SCRH004
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MALAT1,0,0,301,205,383,405,0,0,1,0,332,289,179,158
DLGAP1,0,0,217,257,299,291,0,0,0,0,120,162,92,55
NEAT1,0,0,114,34,62,40,0,0,0,0,15,48,38,36
PBX3,0,0,74,12,55,18,0,0,0,0,14,3,22,8
BCL2,0,0,73,56,48,11,0,0,0,0,28,7,16,2
CTC-340A15.2,0,0,70,2,34,4,0,0,0,0,3,0,0,2
PARD3,0,0,67,42,34,25,0,0,0,0,23,3,7,2
NCALD,0,0,64,7,55,68,0,0,0,0,10,16,1,11
CTNNA3,0,0,61,3,80,7,0,0,0,0,0,65,11,2


In [31]:
RNAres <- read_table2("./result/RNA_Res/counts.gene.tsv")


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  `CRH1-1` = col_double(),
  `CRH1-2` = col_double(),
  `CRH1-3` = col_double(),
  `CRH1-4` = col_double(),
  `CRH1-5` = col_double(),
  `CRH1-6` = col_double(),
  `HC1-1` = col_double(),
  `HC1-2` = col_double(),
  `HC1-6` = col_double()
)




In [36]:
names(RNAres)[1:4] <- c("gene","cell1","cell2","cell3")
RNAres %>% arrange(desc(cell2))

gene,cell1,cell2,cell3,CRH1-4,CRH1-5,CRH1-6,HC1-1,HC1-2,HC1-6
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DLGAP1,22,48,31,26,13,1,0,0,0
MALAT1,18,23,56,16,42,1,0,0,0
AUTS2,4,22,23,1,11,1,0,0,0
PTPRG,12,12,4,7,1,0,0,0,0
DOCK10,6,11,3,0,2,0,0,0,0
KCND2,9,11,15,16,15,0,0,0,0
AC096559.1,4,10,2,10,0,0,0,0,0
CUX1,3,10,5,2,4,3,0,0,0
ATXN1,8,9,7,1,6,4,0,0,0


In [6]:
# calculate total count of RNA
print("total counts")
colSums(RNAres %>% select(-gene))
# calculate differnet types of RNA
print("total types")
colSums(RNAres %>% select(-gene) != 0)

[1] "total counts"


CRH1-1 CRH1-2 CRH1-3 CRH1-4 CRH1-5 CRH1-6  HC1-1  HC1-2  HC1-6 
  3532   3608   3393   3378   3784   1795      2      1      1

[1] "total types"


CRH1-1 CRH1-2 CRH1-3 CRH1-4 CRH1-5 CRH1-6  HC1-1  HC1-2  HC1-6 
  2059   2149   1981   1884   2078   1196      2      1      1

In [4]:
dim(stat)

[1]  8 11

In [3]:
plot <- stat %>% mutate(RNAreadsRatio = RNAreads/(RNAreads+DNAreads),
                        pairsPerRead = raw_pairs/DNAreads,
                        pairsValidRatio = pairs_clean2/raw_pairs,
                        interPairsRatio = inter_pairs_clean2/pairs_clean2) 
#%>% filter(str_detect(cell_name,"R"))


In [4]:
plot

,cell_name,DNAreads,Rawreads,RNAreads,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,inter_pairs_clean2,UMIs_gene,genes_gene,RNAreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
B7001,B7001,45679622,48944497,2852542,8050169,830247,815096,710891,60663,45300,8017,0.0587763200,0.1762311,0.08830759,0.08533376
B7002,B7002,38361319,38443622,16440,5802569,628778,608845,515796,75612,256,195,0.0004283731,0.1512609,0.08889097,0.14659284
B7003,B7003,35113721,36089580,823204,5876246,649285,643904,568106,68688,14419,4560,0.0229069126,0.1673490,0.09667839,0.12090701
B7004,B7004,39558507,40735070,920339,4720672,666082,654598,582605,76103,19949,5369,0.0227362954,0.1193339,0.12341569,0.13062538
B7005,B7005,43640125,43748423,26849,7512533,976979,956008,828086,178612,410,304,0.0006148583,0.1721474,0.11022727,0.21569257


In [5]:
plot %>% write_tsv(file="~/test/202106/statForDifferentCelltype/brain10.tsv")

In [13]:
RNAres %>% filter(str_detect(gene,"MT"))

gene,B7001,B7002,B7003,B7004,B7005
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ADAMTS12,0,0,0,1,0
ADAMTS13,1,0,1,0,0
ADAMTS16,0,0,5,2,0
ADAMTS19,0,0,0,3,0
ADAMTS2,1,0,1,0,0
ADAMTS20,2,0,0,0,0
ADAMTS3,12,0,0,2,0
ADAMTS6,0,0,0,1,0
ADAMTS7,1,0,0,0,0
